The goal of this notebook is to stack the bias frames to create a main bias for each observation date. We also aim to assess whether any frames differ significantly.

In [ ]:
foldername = ["20230708", "20230709", "20230723", "20230816"][2]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ccdproc
from ccdproc import ImageFileCollection

from astropy.nddata import CCDData
import arya

In [ ]:
from convenience_functions import show_image, combine_images, show_image_residual, mad_std

In [ ]:
def get_biases(foldername):
    imgfiles = ImageFileCollection(foldername, glob_include="bias*.fits")

    return imgfiles

In [ ]:
imgfiles = get_biases(foldername + "/trimmed/")

In [ ]:
bias_stacked = CCDData.read(foldername + "/bias_stacked.fits")

In [ ]:
bias_std = CCDData.read(foldername + "/bias_stacked_err.fits")

# Images

In [ ]:
%matplotlib inline


In [ ]:

for ccd, fname in imgfiles.ccds(return_fname=True):
    show_image(ccd, dpi=100, figsize=(5, 5))
    plt.title(fname)

In [ ]:
for ccd, fname in imgfiles.ccds(return_fname=True):
    show_image_residual(ccd.data , bias_stacked.data, dpi=100, figsize=(5, 5), clim=(-10, 10))
    plt.title(fname)


In [ ]:
show_image(bias_std)

In [ ]:
show_image(bias_stacked)

# Do any pixels have poorly-behaved fluctuations

In [ ]:
noise = np.median(bias_std)

In [ ]:
noise

In [ ]:
plt.figure()
plt.hist(np.log10(np.maximum(bias_std.data.flatten(), 1e-3)))
plt.yscale("log")
plt.axvline(np.log10(noise), color="black")
plt.xlabel("log std(pixel value)")

In [ ]:
bad_pixel_count = np.zeros_like(bias_stacked)

for ccd, fname in imgfiles.ccds(return_fname=True):
    bad_pixels = np.abs(ccd.data - bias_stacked.data) / noise > 5
    bad_pixel_count += bad_pixels
    print(np.sum(bad_pixels), "\t", np.mean(bad_pixels))


In [ ]:
np.max(bad_pixel_count)

In [ ]:
assert np.max(bad_pixel_count) == 1

In [ ]:
np.sum(bad_pixel_count > 1)

In [ ]:
plt.figure()
plt.hist(bad_pixel_count.flatten())

In [ ]:
%matplotlib ipympl

show_image(bad_pixel_count, clim=(0, 2))

In [ ]:
np.where(bad_pixel_count > 1)

In [ ]:
1532, 900 # (bad pixel last)